In [1]:
from math import log


def remove(data: list, axis: int):
    _1 = data[0:axis]
    _2 = data[axis + 1:len(data)]
    _1.extend(_2)
    return _1


def majority_value(data_set, target_attr):
    """
    Majority values from data_set with target_attr.
    """
    # Count labels from data set
    labels_dict = {}
    values_list = []

    for i in [c[-1] for c in data_set]:
        # select last node (same attribute) in data set
        if i not in values_list:
            # record the values that could be, like 0, 1, 2, 3 or 17
            # (17 is ID and was deleted already. 0, 1 is colors or something else)
            values_list.append(i)
    # each line in data
    for line in data_set:
        # select each target attribute in line, in number is like 1, 2, 3. In Chinese is 青绿.
        label = line[target_attr]
        # if this label not in dict, create and + 1.
        labels_dict[label] = labels_dict.get(label, 0) + 1

    for i in range(len(values_list) - 1):
        # branching according to the number of corresponding attributes
        if labels_dict[values_list[i]] < labels_dict[values_list[i + 1]]:
            values_list.pop(i)
        else:
            values_list.pop(i + 1)
    # returns the classification result of the most likely corresponding attribute
    return values_list[0]


def get_values(dataset, best_attribute, attributes):
    index = attributes.index(best_attribute)
    tmp_values = []
    for a in [c[index] for c in dataset]:
        if a not in tmp_values:
            tmp_values.append(a)
    return tmp_values


def get_shannon_ent(sub_data_set):
    num_entries = len(sub_data_set)
    labels_dict = {}

    for line in sub_data_set:
        label = line[-1]
        labels_dict[label] = labels_dict.get(label, 0) + 1

    shannon_ent = 0.0

    for key in labels_dict:
        prob = float(labels_dict[key]) / num_entries
        shannon_ent -= prob * log(prob, 2)

    return shannon_ent


def get_examples(data_set, axis, value):
    ret_data_set = []
    for feat_vec in data_set:
        if feat_vec[axis] == value:
            reduced_feat_vec = feat_vec[:axis]
            reduced_feat_vec.extend(feat_vec[axis + 1:])
            ret_data_set.append(reduced_feat_vec)
    return ret_data_set


def choose_attribute(data_set, attributes):
    num_features = len(data_set[0]) - 1
    base_entropy = get_shannon_ent(data_set)
    best_info_gain = 0.0
    best_feature = -1

    for i in range(num_features):
        feat_list = [example[i] for example in data_set]
        unique_vals = set(feat_list)
        new_entropy = 0.0

        for value in unique_vals:
            sub_data_set = get_examples(data_set, i, value)
            prob = len(sub_data_set) / float(len(data_set))
            new_entropy += prob * get_shannon_ent(sub_data_set)
        info_gain = base_entropy - new_entropy
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = attributes[i]
    return best_feature


def create_decision_tree(data, attributes, target_attr, call_count, source_attributes_num):
    call_count += 1
    print("call_count:", call_count)
    data = data[:]
    vals = [record[target_attr] for record in data]
    default = majority_value(data, target_attr)
    # If the dataset is empty or the attributes list is empty, return the
    # default value. When checking the attributes list for emptiness, we
    # need to subtract 1 to account for the target attribute.
    if not data or (len(attributes)) <= 0:
        return default
    # If all the records in the dataset have the same classification,
    # return that classification.
    elif vals.count(vals[0]) == len(vals):
        return vals[0]
    else:
        # Choose the next best attribute to best classify our data
        best = choose_attribute(data, attributes)
        # Create a new decision tree/node with the best attribute and an empty
        # dictionary object--we'll fill that up next.
        tree = {best: {}}
        # Create a new decision tree/sub-node for each of the values in the
        # best attribute field
        test_val = get_values(data, best, attributes)
        for i in source_attributes_num[attributes.index(best)]:
            if i not in test_val:
                tree[best][i] = default

        for val in test_val:
            # Create a subtree for the current value under the "best" field
            subtree = create_decision_tree(get_examples(data, attributes.index(best), val),
                                           [attr for attr in attributes if attr != best],
                                           target_attr, call_count,
                                           remove(source_attributes_num, attributes.index(best)))
            # Add the new subtree to the empty dictionary object in our new
            # tree/node we just created.
            tree[best][val] = subtree
    return tree


In [2]:
if __name__ == "__main__":
    # read data
    import csv

    csv_reader = csv.reader(open("wm3.csv"))

    # remove the first ID line (preventing the over-fitting problem)
    source_data = [row[1:] for row in csv_reader]

    # attributes in Chinese
    attributes_labels = ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感']

    # get attributes in number
    attributes_nums = []
    for y in range(len(source_data[0]) - 1):
        tmp = []
        for x in range(len(source_data)):
            if source_data[x][y] not in tmp:
                tmp.append(source_data[x][y])
        attributes_nums.append(tmp)

    # generate predict data
    prediction = create_decision_tree(source_data, attributes_labels, -1, 0, attributes_nums)

    # store prediction result as dict string
    with open('prediction.txt', 'w', encoding="utf-8") as dict_file:
        dict_file.write(str(prediction))

call_count: 1
call_count: 2
call_count: 3
call_count: 3
call_count: 4
call_count: 4
call_count: 5
call_count: 5
call_count: 3
call_count: 2
call_count: 3
call_count: 3
call_count: 2
